In [19]:
from gensim.models import Word2Vec

In [37]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [30]:
train = pd.read_csv("data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("data/testData.tsv", header=0, delimiter="\t", quoting=3)

In [20]:
model = Word2Vec.load("300features_40minwords_10context")

In [21]:
type(model.syn0)

numpy.ndarray

In [22]:
model.syn0.shape

(19367, 300)

In [39]:
num_features = 300

In [23]:
model["flower"]

array([  3.88699844e-02,  -5.79281636e-02,  -4.72524166e-02,
        -3.44215110e-02,  -5.52127138e-02,  -3.30933109e-02,
         3.13735828e-02,   1.38976257e-02,   2.75714416e-03,
         5.81019185e-02,  -6.12710081e-02,   6.58257753e-02,
         1.07289098e-01,   3.88932088e-03,  -1.03177443e-01,
        -6.23398228e-03,   5.54352216e-02,   1.23001933e-01,
         4.97372970e-02,   3.10368817e-02,   5.95951639e-02,
        -1.90753508e-02,   8.29407126e-02,  -2.54650749e-02,
         7.09344121e-03,   2.69894898e-02,   2.25821976e-02,
        -1.82786044e-02,   7.49888346e-02,  -1.27652232e-02,
         6.71777055e-02,  -3.18174139e-02,   7.71801025e-02,
         1.05839381e-02,   1.87179428e-02,   5.39638568e-03,
        -3.91476937e-02,  -7.63301796e-04,   4.01199162e-02,
        -9.08813905e-03,  -9.09716189e-02,   4.34762612e-02,
         8.72895941e-02,   2.94513088e-02,   5.69718517e-02,
         8.08170959e-02,  -1.61818322e-02,   1.38613908e-02,
         1.01348162e-01,

#### From Words To Paragraphs, Attempt 1: Vector Averaging

In [24]:
import numpy as np

In [32]:
def review_to_wordlist(review, remove_stopwords=False):
    # Remove HTML
    review_text = BeautifulSoup(review).get_text()
    # Remove non-letters
    review_text = re.sub("[^a-zA-Z]", " ", review_text)
    # Convert words to lower case and split them
    words = review_text.lower().split()
    # Remove stopwords
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
        
    return words

In [25]:
def makeFeatureVec(words, model, num_features):
    '''
    Creates average feature vector given a list of words
    '''
    # initialize empty vector
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    # list of words as set
    index2word_set = set(model.index2word)
    # iterate over each word and add up associated feature vectors
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [26]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
        if counter%1000. == 0.:
            print("Review %d of %d" % (counter, len(reviews)))
        # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        # Increment the counter
        counter = counter + 1.
    return reviewFeatureVecs

In [40]:
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features)

/Users/Christian/anaconda/envs/dlmovies/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/Christian/anaconda/envs/dlmovies/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 o

In [42]:
trainDataVecs.shape

(25000, 300)

#### Create Random Forest

In [43]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

In [44]:
forest = forest.fit( trainDataVecs, train["sentiment"] )

In [45]:
result = forest.predict( testDataVecs )

In [46]:
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

In [54]:
# calculate accuracy
np.sum(result == train["sentiment"])/len(result)

0.49764000000000003

#### From Words to Paragraphs, Attempt 2: Clustering 

In [ ]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.syn0
num_clusters = round(word_vectors.shape[0] / 5)

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start

In [ ]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.index2word, idx ))

In [ ]:
# Checkout clusters

In [ ]:
# For the first 10 clusters
for cluster in range(0,10):
    #
    # Print the cluster number  
    print("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words

In [ ]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

so basically a "bag of centroids" is a vector of length "number of clusters". we loop over each word in each review and increment the cluster indicator by one if we have a hit, i.e. a certain word is from a certain cluster. in the end each review is represented as a vector of cluster counts

In [ ]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), 
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, 
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), 
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, 
        word_centroid_map )
    counter += 1
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print "Fitting a random forest to labeled training data..."
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )